# Converting traces in raw format

In this prerequisite part, we will first change the trace format, from the npy generated by the capture, to a "raw" format, simply consisting of writing the different values measured consecutively. Of course, the number of samples in one trace and the number of traces must be kept alongside the file (for example in a header file with #define).
First, let us make a capture.

In [6]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
CRYPTO_TARGET = 'TINYAES128C'

In [7]:
%run "Helper_Scripts/Setup_Generic.ipynb"

In [8]:
fw_path = "../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex".format(PLATFORM)

In [9]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2

rm -f -- simpleserial-aes-CWLITEXMEGA.hex
rm -f -- simpleserial-aes-CWLITEXMEGA.eep
rm -f -- simpleserial-aes-CWLITEXMEGA.cof
rm -f -- simpleserial-aes-CWLITEXMEGA.elf
rm -f -- simpleserial-aes-CWLITEXMEGA.map
rm -f -- simpleserial-aes-CWLITEXMEGA.sym
rm -f -- simpleserial-aes-CWLITEXMEGA.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- simpleserial-aes.s simpleserial.s XMEGA_AES_driver.s uart.s usart_driver.s xmega_hal.s aes.s aes-independant.s
rm -f -- simpleserial-aes.d simpleserial.d XMEGA_AES_driver.d uart.d usart_driver.d xmega_hal.d aes.d aes-independant.d
rm -f -- simpleserial-aes.i simpleserial.i XMEGA_AES_driver.i uart.i usart_driver.i xmega_hal.i aes.i aes-independant.i
.
-------- begin --------
avr-gcc (GCC) 4.9.2
Copyright (C) 2014 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling C: simpleserial-aes.c
avr-gcc -c -mmcu=atxme

In [10]:
cw.program_target(scope, prog, fw_path)

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 3985 bytes


In [11]:
# Capture Trace
import time

ktp = cw.ktp.Basic()

key, text = ktp.next()
traces = []

nb_samples = 3000
scope.adc.samples = nb_samples
scope.adc.offset = 0

trace = cw.capture_trace(scope, target, text, key)
traces.append(trace)

We will now print the first 10 values and write the trace to a file in "raw" format.

In [37]:
import numpy as np
import os

traces_dir = 'traces'
#os.mkdir(traces_dir)
trace_array = np.asarray([trace.wave for trace in traces])
print(trace_array)

for i in range(10):
    print('%f' % trace_array[0][i])

# Write trace to file
f = open(os.path.join(traces_dir, 'traces.raw'), "wb")
data = trace_array.astype('float64')

f.write(data)
f.close()

[[ 0.08691406 -0.31054688 -0.1328125  ... -0.17773438 -0.02441406
  -0.04785156]]
0.086914
-0.310547
-0.132812
-0.131836
0.021484
-0.412109
-0.208984
-0.193359
-0.024414
-0.472656


In [40]:
from tqdm import tnrange
traces = [] # list of traces
N = 5000  # Number of traces

for i in tnrange(N, desc = 'Capturing traces'):
    key, text = ktp.next()  # creation of a pair comprising (fixed) key and text 

    trace = cw.capture_trace(scope, target, text, key) # a trace is composed of the following fields :
                                                       #    a wave (samples)
                                                       #    textin (input text), textout (output text)
                                                       #    key (input key)
    
    if trace is None:
        continue
    traces.append(trace)

# Convert traces to numpy arrays
trace_array = np.asarray([trace.wave for trace in traces])  # if you prefer to work with numpy array for number crunching
textin_array = np.asarray([trace.textin for trace in traces])
known_keys = np.asarray([trace.key for trace in traces])    # for fixed key generation, these keys are all the same

In [42]:
traces_dir = 'traces'
os.mkdir(traces_dir)

f = open(os.path.join(traces_dir, 'traces.raw'), "wb")
f2 = open(os.path.join(traces_dir, 'plaintexts.raw'), "wb")
f3 = open(os.path.join(traces_dir, 'knownkeys.raw'), "wb")

data = trace_array.astype('float64')
f.write(data)
f.close()

data = textin_array.astype('uint8')
f2.write(data)
f2.close()

data = known_keys.astype('uint8')
f3.write(data)
f3.close()

In [46]:
#test 
test_f = open(os.path.join(traces_dir, 'traces.raw'), "rb")

d = np.fromfile(test_f, 'float64')

print(d[:10])
print(d[3000*4999: 3000*4999+10])

test_f.close()

[ 0.08984375 -0.31054688 -0.13183594 -0.13183594  0.02246094 -0.41308594
 -0.20996094 -0.19238281 -0.02636719 -0.47070312]
[ 0.08789062 -0.30957031 -0.13574219 -0.13476562  0.02050781 -0.41894531
 -0.21386719 -0.19628906 -0.02539062 -0.47460938]


You must now write a C program which reads the file into an array and prints the first 10 values, using the following code skeleton, in which path is a buffer containing the file name.

    #define NB_SAMPLES 3000  // should be in a header file !
    
    double traces[NB_SAMPLES];  //only one trace here, to be adapted if needed
    
    if ((fd = open(path, O_RDONLY)) == -1) {
        perror("open");
        return -1;
    }

    if (read(fd, traces, sizeof(double) * NB_SAMPLES) == -1) {
        perror("read");
        return -1;
    }
    
    for (int i = 0; i < 10; i += 1) {
        printf("%f\n", traces[i]);
    }
    

In [51]:
p = [ 43, 126,  21,  22 , 40 ,174 ,210, 166, 171 ,247  ,21 ,136 ,  9, 207,  79 , 60]
print(p)
[print(hex(x)) for x in p]

[43, 126, 21, 22, 40, 174, 210, 166, 171, 247, 21, 136, 9, 207, 79, 60]
0x2b
0x7e
0x15
0x16
0x28
0xae
0xd2
0xa6
0xab
0xf7
0x15
0x88
0x9
0xcf
0x4f
0x3c


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

Verify that you observe the same 10 values than in python. Change the python code above in order to convert the paintext and the key in raw type, using `astype('uint8')`, and write them in separate files.
You can check the content of the raw key file (for example) using either the linux commands `xxd` or `hexdump`.

Modify your script in order to generate raw files for a set of several traces with different random plaintexts. The file containing the traces and the file contaning the plaintext must have a length of `nb_samples * nb_traces * sizeof(double)` and `16 * nb_traces` respectively, which you can check with the linux `ls -l` command.

Note: it is possible that some traces contain an incorrect number of samples. It is advised to check the length of the trace right after its capture, and redo the capture if it is incorrect.

# Implementing CPA

You must now implement a CPA (Correlation Power Analysis) in C, using traces that you have converted as above. It should be much faster to execute in C than in python. You must use the Pearson correlation coefficient as presented during the lectures. 

Your code must be as generic as possible regarding the number of samples, traces, and file names. It should allow to perform the attack on a sub-interval of the traces, by reading into memory only this sub-interval for each trace.

In your report, you must present the results of an evaluation regarding the effect of the trace number on the success rate of the attack (in terms of number of key bytes correctly recovered), and compare it with the DPA results.

Finally, as a bonus, you can implement the DPA attack in C in order to compare the execution time between both attacks, and between the C and the python implementation. Explain the results that you have obtained in your report.